### Cell 1: Import Libraries

This cell imports the necessary libraries for the notebook. It includes:
- `pandas` for data manipulation.
- `arcgis.gis` and `arcgis.features` for interacting with ArcGIS Online.
- `ipywidgets` and `IPython.display` for creating interactive widgets.
- `io` for handling byte streams.
- `requests` for making HTTP requests.

In [1]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import ipywidgets as widgets
from IPython.display import display
import io
import requests


### Cell 2: User Authentication

This cell creates widgets for user input to sign in to ArcGIS Online. It includes:
- A text widget for the username.
- A password widget for the password.
- A button to trigger the sign-in process.

When the button is clicked, the `on_sign_in_button_clicked` function is called, which signs in to ArcGIS Online using the provided credentials and prints a success message.

In [2]:
# Step 1: Create widgets for user input
username_widget = widgets.Text(description="Username:")
password_widget = widgets.Password(description="Password:")

# Step 2: Display widgets
display(username_widget, password_widget)

# Step 3: Function to sign in
def sign_in(username, password):
    return GIS("https://www.arcgis.com", username, password)

# Step 4: Button to trigger sign in
sign_in_button = widgets.Button(description="Sign In")

def on_sign_in_button_clicked(b):
    global gis
    username = username_widget.value
    password = password_widget.value
    gis = sign_in(username, password)
    print("Signed in successfully.")

sign_in_button.on_click(on_sign_in_button_clicked)
display(sign_in_button)

Text(value='', description='Username:')

Password(description='Password:')

Button(description='Sign In', style=ButtonStyle())

### Cell 3: Item ID and CSV File Upload

This cell creates widgets for inputting the item ID of the feature layer and uploading a CSV file. It includes:
- A text widget for the item ID.
- A file upload widget for the CSV file.
- A button to load the layers and create a DataFrame from the uploaded CSV file.

When the button is clicked, the `on_load_layers_button_clicked` function is called, which reads the CSV file, creates a DataFrame, and displays the first few rows of the DataFrame.

In [3]:

# Step 1: Create widgets for item ID and CSV file upload
item_id_widget = widgets.Text(description="Item ID:")
csv_file_widget = widgets.FileUpload(description="Upload CSV", accept='.csv', multiple=False)

# Step 2: Display widgets
display(item_id_widget, csv_file_widget)

# Step 3: Function to read CSV and create DataFrame
def create_dataframe(csv_file):
    return pd.read_csv(io.BytesIO(csv_file))

# Step 4: Button to trigger layer population and DataFrame creation
load_layers_button = widgets.Button(description="Load Layers")

def on_load_layers_button_clicked(b):
    global item_id, df
    item_id = item_id_widget.value
    csv_file = list(csv_file_widget.value.values())[0]['content']
    df = create_dataframe(csv_file)
    print("DataFrame created successfully.")
    display(df.head())

load_layers_button.on_click(on_load_layers_button_clicked)
display(load_layers_button)

Text(value='', description='Item ID:')

FileUpload(value=(), accept='.csv', description='Upload CSV')

Button(description='Load Layers', style=ButtonStyle())

### Cell 4: Layer Selection

This cell creates a dropdown widget for selecting a layer from the item. It includes:
- A dropdown widget for layer selection.
- A button to confirm the layer selection and proceed.

The `populate_layers` function populates the dropdown with the available layers from the item. When the button is clicked, the `on_create_df_button_clicked` function is called, which sets the selected layer and prints the selected layer's name.

In [7]:
# Step 1: Create widget for layer selection
layer_selection_widget = widgets.Dropdown(description="Layer:")

# Step 2: Display widget
display(layer_selection_widget)

# Step 3: Function to populate layer dropdown based on item ID
def populate_layers(item_id):
    item = gis.content.get(item_id)
    if item is None:
        print(f"Item with ID {item_id} not found.")
        return
    layers = item.layers
    layer_options = [(layer.properties.name, layer) for layer in layers]
    layer_selection_widget.options = layer_options

# Step 4: Populate layers after item ID is provided
populate_layers(item_id)

# Step 5: Button to trigger DataFrame creation
create_df_button = widgets.Button(description="Select Layer and Proceed")

def on_create_df_button_clicked(b):
    global selected_layer
    selected_layer = layer_selection_widget.value
    print(f"Selected layer: {selected_layer.properties.name}")

create_df_button.on_click(on_create_df_button_clicked)
display(create_df_button)

Dropdown(description='Layer:', options=(), value=None)

Button(description='Select Layer and Proceed', style=ButtonStyle())

Selected layer: monitoring_system.sde.diem_adm_repr_1_mview


### Cell 5: Column Selection

This cell creates dropdown widgets for selecting the columns for "Field Name" and "Alias" from the DataFrame. It includes:
- A dropdown widget for the "Field Name" column.
- A dropdown widget for the "Alias" column.

The `populate_dropdowns` function populates the dropdowns with the column names from the DataFrame. The dropdowns are displayed for user interaction.

In [11]:
# Step 1: Create dropdown widgets for selecting columns
field_name_column_widget = widgets.Dropdown(description="Field Name Column:")
alias_column_widget = widgets.Dropdown(description="Alias Column:")

# Step 2: Function to populate dropdowns
def populate_dropdowns(df):
    columns = df.columns.tolist()
    field_name_column_widget.options = columns
    alias_column_widget.options = columns

# Step 3: Populate dropdowns after DataFrame is created
populate_dropdowns(df)

# Step 4: Display dropdowns
display(field_name_column_widget, alias_column_widget)


Dropdown(description='Field Name Column:', options=('Variable Groups', 'Field Name', 'Alias', 'Thematic area (…

Dropdown(description='Alias Column:', options=('Variable Groups', 'Field Name', 'Alias', 'Thematic area (Disag…

### Cell 6: Update Field Aliases

This cell updates the field aliases of the selected layer based on the values in the DataFrame. It includes:
- Retrieving the current fields of the selected layer.
- Converting the fields to a list of dictionaries.
- Iterating through the fields and updating the aliases based on the DataFrame.
- Updating the feature layer with the new field aliases using the `update_definition` method.
- Printing a success message after the aliases are updated.

In [13]:
# Get the current fields
fields = selected_layer.properties.fields

# Convert fields to a list of dictionaries
fields_dict = [dict(field) for field in fields]

# Update the field aliases based on the CSV file
for field in fields_dict:
    field_name = field['name']
    if field_name in df[field_name_column_widget.value].values:
            alias = df[df[field_name_column_widget.value] == field_name][alias_column_widget.value].values[0]
            field['alias'] = alias

# Update the feature layer with the new field aliases
selected_layer.manager.update_definition({'fields': fields_dict})
print("Field aliases updated successfully.")

Field aliases updated successfully.
